# CNN

This notebook uses FastAI, a PyTorch-based deep learning library, to build the neural network, which is able to figure out the relationship between input features and find hidden relationship with them. The input data is an image dataset with labels, which is converted from time series with Gramian Angular Field algorithm as described in the previous sections.  

In [1]:
# import libraries and packages
import torch
from fastai.vision import *
from fastai.metrics import error_rate
from fastai.tabular import *
import pandas as pd

In [2]:
torch.__version__

'1.7.1'

In [3]:
torch.cuda.is_available()

True

In [4]:
torch.cuda.set_device(0)

In [5]:
gramian_df = pd.read_csv('../data/processed data/gramian_df.csv', index_col='date')

In [6]:
path = '../data/processed data'
df = pd.read_csv(path+'/label_dir_2.csv')
df = df[df.date.isin(list(gramian_df.index))]
df.date = df.date.apply(lambda x: x+'.png')
df.head()

,date,label
0,2017-01-02.png,0
1,2017-01-03.png,1
2,2017-01-04.png,0
3,2017-01-05.png,1
4,2017-01-06.png,0


In [7]:
df.groupby('label').date.count()

label
0    531
1    457
Name: date, dtype: int64

In [8]:
data = ImageDataBunch.from_df(path, df, valid_pct=0.2, size=224)

NameError: name 'ImageDataBunch' is not defined

In [ ]:
data.show_batch(rows=3, figsize=(6,6))

### 1. Base CNN (ResNet34 + Custom Head)

For the CNN network, the pretrained ResNET-34 is utilized as the bottom layers. We added [1024, 2] dense layers on top and a simple linear activation node for the final regression as a custom head. The figure below shows the architecture of the top layers of the model. For the loss function, our final model used cross-entropy loss. For the model metrics, we used accuracy. 

In [ ]:
# Create custom_head
layers = [AdaptiveConcatPool2d(), Flatten(), nn.Linear(1024,2)]
head = nn.Sequential(*layers)

In [ ]:
acc_02 = partial(accuracy_thresh, thresh=0.2)
f_score = partial(fbeta, thresh=0.2)

learner = cnn_learner(data, 
                      models.resnet34, 
                      pretrained=True,
                      custom_head=head,
#                       loss_func=mae,
                     metrics=[error_rate, accuracy]
#                       metrics=[acc_02, f_score]
                     ).to_fp16()
#learner = cnn_learner(data, models.resnet34, pretrained = True, custom_head=head, loss_func=mae, metrics=[r2_score]).to_fp16()
learner.model[1]

In [ ]:
learner.lr_find(start_lr=1e-07,end_lr=0.2, num_it=100)
learner.recorder.plot(suggestion=True)

In [ ]:
mingradlr = learner.recorder.min_grad_lr
print(mingradlr)

In [ ]:
lr = mingradlr
learner.fit_one_cycle(10, lr)

In [ ]:
learner.unfreeze()
learner.lr_find(start_lr=1e-07,end_lr=0.2, num_it=100) 
learner.recorder.plot(suggestion=True)

In [ ]:
mingradlr1 = learner.recorder.min_grad_lr
print(mingradlr1)

In [ ]:
learner.fit_one_cycle(10, slice(mingradlr1, mingradlr1/20))

In [ ]:
learner.show_results(rows=3)

In [ ]:
interp = ClassificationInterpretation.from_learner(learner)
interp.plot_confusion_matrix(figsize=(5,5))

In [ ]:
learner.save('model_1')

## Grad-CAM

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from importlib import reload

In [ ]:
import gradcam
reload(gradcam)
from gradcam import *

In [ ]:
test_img = '../data/imgs/2017-01-03.png'
img = open_image(test_img);

In [ ]:
img.show()

In [ ]:
layers = [AdaptiveConcatPool2d(), Flatten(), nn.Linear(1024,2)]
head = nn.Sequential(*layers)

gramian_df = pd.read_csv('../data/processed data/gramian_df.csv',index_col='date')
path = '../data/processed data'
df = pd.read_csv(path+'/label_dir_2.csv')
df = df[df.date.isin(list(gramian_df.index))]
df.date = df.date.apply(lambda x: x+'.png')
data = ImageDataBunch.from_df(path, df, valid_pct=0.2, size=224)

learner = cnn_learner(data, 
                      models.resnet34, 
                      pretrained=True,
                      custom_head=head,
                     metrics=[error_rate, accuracy]
                     ).to_fp16()

In [ ]:
learner.load('model_1');

In [ ]:
# gcam = GradCam.from_one_img(learner,img)
# gcam.plot()